In [31]:
### Import libraries
import requests
from bs4 import BeautifulSoup
import time 
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [27]:
def player_info_scrape(source_code, player, player_id = None):
    '''
    Scrapes birthday, height, weight, place of birth, college, high school (Name and State), and draft info from profootballreference.com 
    :param source_code: is the browser page source that is accessed through selenium 
    :type source_code: str
    :param player: is the player's name that we are scraping for 
    :type player: str
    :return: None
    '''
    play_page_soup = BeautifulSoup(source_code, 'html.parser')
    try: 
        birthday_tag = play_page_soup.find('span"', {"id":"necro-birth"})
        birthday = birthday_tag['data-birth']
    except:
        birthday = "NA"
        print(f'{player} not found')
        naplayers.append(player)
        return

    try: 
        height = play_page_soup.find("strong", string = "Position").find_next("p").find_next("span").get_text()
        height_split = height.split("-")
        feet = int(height_split[0])
        inches = int(height_split[1])
        
    except:
        feet = "NA"
        inches = "NA"
        
    try: 
        weight_1 = play_page_soup.find("strong", string = "Position").find_next("p").find_next("span").find_next("span").string
        weight = weight_1.rstrip("lb")
    except:
        weight = "NA"
        
    try: 
        born = play_page_soup.find('span"', {"id":"necro-birth"}).find_next("span").get_text()

    except:
        born = "NA"

    try: 
        college = play_page_soup.find("strong", string = "College").find_next("a").string
    except:
        college = "NA"
    try: 
        high_school = play_page_soup.find("strong", string = "High School")
        high_school_name = high_school.find_next("a").string
        high_school_state = high_school.find_next("a").find_next("a").string

    except:
        high_school_name = "NA"
        high_school_state = "NA"

    try: 
        draft_team = play_page_soup.find("strong", string = "Draft").find_next("a").string
    except:
        draft_team = "NA"
    try: 
        draft_info = play_page_soup.find("strong", string = "High School").find_next("p").get_text()

    except:
        draft_info = "NA"


        
    print(birthday, feet, inches, weight, born, college, high_school_name , high_school_state, draft_team, draft_info)
    if player_id:
        data.append((player_id, player, birthday, feet, inches, weight, born, college, high_school_name , high_school_state, draft_team, draft_info))
    else:
        data.append(( player, birthday, feet, inches, weight, born, college, high_school_name , high_school_state, draft_team, draft_info))
        
    time.sleep(5)
    



In [21]:
##Get list of player names
player_names = pd.read_csv("ffball_project_data_2.csv")
player_names = player_names["player_name"]
player_names

0       A.J. Brown
1       A.J. Brown
2       A.J. Brown
3       A.J. Brown
4       A.J. Brown
           ...    
8709     Zay Jones
8710     Zay Jones
8711     Zay Jones
8712     Zay Jones
8713     Zay Jones
Name: player_name, Length: 8714, dtype: object

In [22]:
###Create set to have all unique values and order them in alphabetical order
s_player_names = set()
for name in player_names: 
    s_player_names.add(name)
player_names = list(s_player_names)
player_names.sort()

In [33]:
### First round of scraping 
data = []
naplayers = []
#go to website
chrome_path= r"*****************"

service_obj = Service(chrome_path)
browser = webdriver.Chrome(service = service_obj)
url = "https://www.pro-football-reference.com/"

browser.get(url)
time.sleep(3)

#list to iterate through the players
#Note: indexing may make this easier to do over time as it is computationally expensive to scrape all at once
for player in player_names[:5]:
  #finds search bar
    try:
        search_bar = browser.find_element(By.CSS_SELECTOR, "input[type='search'][class='ac-input completely']")
    except: 
        search_bar = browser.find_element(By.CSS_SELECTOR, "input[type='search'][class='ac-input completely prefilled']")
    search_bar.clear()
    time.sleep(1.5)
    #types in player name
    search_bar.send_keys(player)
    time.sleep(1.5)
    #clicks the search button
    search_button = browser.find_element(By.CSS_SELECTOR, "input[value = 'Search'][tabindex = '2']")
    try:
        search_button.click()
    #clicks out of the pop up add
    except:
        pop_up_add = browser.find_element(By.CSS_SELECTOR, "div[id = 'modal-close'][aria-label = 'Close in-page popup window']")
        pop_up_add.click()
        search_button.click()

        
    time.sleep(1.5)
    
    source_code = browser.page_source
    
    player_info_scrape(source_code, player)


df = pd.DataFrame(np.array(data))
na_df = pd.DataFrame(np.array(naplayers))
df.columns = "player_name birthday feet inches weight born college high_school_name high_school_state draft_team draft_info".split()
na_df.columns = ["player_name"]
print(df)
if player_names[0] == df.player_name[0]:
    df.to_csv("player_info_scrape.csv", mode = 'w', header = True, index = False)
    na_df.to_csv("na_scrape.csv", mode = 'w', header = True, index = False)
else:
    df.to_csv("player_info_scrape.csv", mode = 'a', header = False, index = False)
    na_df.to_csv("na_scrape.csv", mode = 'a', header = False, index = False)


browser.quit()

1997-06-30 6 1 226 
    in Starkville, MS Mississippi Starkville MS Tennessee Titans Draft: Tennessee Titans in the 2nd round (51st overall) of the 2019 NFL Draft.
A.J. Dillon not found
1977-05-16 6 3 220 
    in Caldwell, ID Oregon Ontario OR Philadelphia Eagles Draft: Philadelphia Eagles in the 5th round (155th overall) of the 2001 NFL Draft.
A.J. Green not found
1976-03-24 6 4 205 
    in Newport News, VA Virginia Ferguson VA Green Bay Packers Draft: Green Bay Packers in the 4th round (131st overall) of the 1999 NFL Draft.
    player_name    birthday feet inches weight                       born  \
0    A.J. Brown  1997-06-30    6      1    226    \n    in Starkville, MS   
1   A.J. Feeley  1977-05-16    6      3    220      \n    in Caldwell, ID   
2  Aaron Brooks  1976-03-24    6      4    205  \n    in Newport News, VA   

       college high_school_name high_school_state           draft_team  \
0  Mississippi       Starkville                MS     Tennessee Titans   
1       Ore

In [ ]:
### For second round of scraping, check the na list. 
### Run the first block of code that uses Selenium and pull the group of players
###Manually go through these and check if the information pulled makes sense the website and get the player links. 
### Watch for players with the same name and add them to a separate list.
### I aleady provide the data for this.

In [14]:
#players = pd.read_csv("player_info_og_na.csv")
duplicates = pd.read_csv("duplicate_player_links.csv")
links = pd.read_excel("linksforna.xlsx")

In [17]:
### This block is for the players whos link we pulled manually
data = []


for i in range(len(links)):
    player = links['player_name'][i]
    link = links['link'][i]

    
    # Check if link is a valid URL
    if link.startswith('http'):
        # Retrieve HTML content from the response
        source_code = requests.get(link)
        
        # Check if the request was successful
        if source_code.status_code == 200:
            source_code = source_code.text
            player_info_scrape(source_code, player)


        else:
            print(f"Failed to fetch {link}, status code: {source_code.status_code}")
    else:
        print(f"Invalid URL: {link}") 
        
df = pd.DataFrame(np.array(data))

df.columns = "player_name birthday feet inches weight born college high_school_name high_school_state draft_team draft_info".split()
print(df)
df.to_csv("player_info_scrape.csv", mode = 'a', header = False, index = False)

1998-05-02 6 0 247 
    in Baltimore, MD Boston Col. Lawrence Academy MA Green Bay Packers Draft: Green Bay Packers in the 2nd round (62nd overall) of the 2020 NFL Draft.
1988-07-31 6 4 207 
    in Summerville, SC Georgia Summerville SC Cincinnati Bengals Draft: Cincinnati Bengals in the 1st round (4th overall) of the 2011 NFL Draft.
1994-12-02 5 9 208 
    in Savannah, GA Texas-El Paso Burges TX Green Bay Packers Draft: Green Bay Packers in the 5th round (182nd overall) of the 2017 NFL Draft.
1988-06-23 6 0 220 
    in Portland, OR Hawaii Benson Polytechnic OR Green Bay Packers Draft: Green Bay Packers in the 3rd round (96th overall) of the 2011 NFL Draft.
1970-05-01 6 1 220 
    in Pittsburgh, PA Pittsburgh Winston Churchill TX Pittsburgh Steelers Draft: Pittsburgh Steelers in the 8th round (216th overall) of the 1993 NFL Draft.
1986-12-15 6 0 224 
    in Baltimore, MD North Carolina St. J.H. Rose NC New York Giants Draft: New York Giants in the 4th round (129th overall) of the 2009 

1991-06-15 5 10 205 
    in Danville, VA Virginia Tech George Washington VA New York Giants Draft: New York Giants in the 1st round (32nd overall) of the 2012 NFL Draft.
1998-10-21 6 4 225 
    in Atlanta, GA Stanford Greater Atlanta Christian GA Houston Texans Draft: Houston Texans in the 3rd round (67th overall) of the 2021 NFL Draft.
1999-02-18 5 11 216 
    in Atlanta, GA Duke Pace Academy GA NA 2023
1991-03-28 6 3 215 
    in Fresno, CA Fresno St. Bakersfield Christian CA Oakland Raiders Draft: Oakland Raiders in the 2nd round (36th overall) of the 2014 NFL Draft.
1971-11-06 6 2 195 
    in Detroit, MI Michigan Benedictine MI Cleveland Browns Draft: Cleveland Browns in the 1st round (29th overall) of the 1994 NFL Draft.
1987-04-11 5 11 210 
    in Atlantic Highlands, NJ Connecticut Red Bank Catholic NJ Indianapolis Colts Draft: Indianapolis Colts in the 1st round (27th overall) of the 2009 NFL Draft.
1987-12-17 6 0 214 
    in Plainfield, NJ Youngstown St. Plainfield NJ NA Twitter

1996-04-22 6 0 200 
    in Chicago, IL Northwestern Glenbard North IL Los Angeles Chargers Draft: Los Angeles Chargers in the 7th round (251st overall) of the 2018 NFL Draft.
1995-04-04 6 2 215 
    in Bridgeville, PA Pennsylvania South Fayette PA Tampa Bay Buccaneers Draft: Tampa Bay Buccaneers in the 5th round (144th overall) of the 2018 NFL Draft.
1999-07-08 5 9 178 
    in Pontiac, MI Penn St. IMG Academy FL Denver Broncos Draft: Denver Broncos in the 2nd round (46th overall) of the 2020 NFL Draft.
1983-07-21 6 4 240 
    in San Diego, CA Miami (FL) Scripps Ranch CA Cleveland Browns Draft: Cleveland Browns in the 1st round (6th overall) of the 2004 NFL Draft.
1976-07-15 5 11 195 
    in Trenton, NJ Syracuse Hamilton West NJ Cleveland Browns Draft: Cleveland Browns in the 2nd round (32nd overall) of the 1999 NFL Draft.
1986-12-17 6 0 210 
    in Miami, FL Central Florida Miami Southridge FL Detroit Lions Draft: Detroit Lions in the 3rd round (64th overall) of the 2008 NFL Draft.
198

1987-11-08 6 4 224 
    in Oklahoma City, OK Oklahoma Putnam City North OK St. Louis Rams Draft: St. Louis Rams in the 1st round (1st overall) of the 2010 NFL Draft.
1997-07-31 5 9 174 
    in Barrington, IL Bowling Green Barrington IL Tampa Bay Buccaneers Draft: Tampa Bay Buccaneers in the 6th round (208th overall) of the 2019 NFL Draft.
1975-11-07 6 4 250 
    in Chickasha, OK Oklahoma Chickasha OK Washington Redskins Draft: Washington Redskins in the 2nd round (48th overall) of the 1998 NFL Draft.
1986-11-28 6 2 210 
    in San Francisco, CA Kentucky Rodriguez CA Buffalo Bills Draft: Buffalo Bills in the 7th round (224th overall) of the 2008 NFL Draft.
1983-07-22 6 2 240 
    in Las Vegas, NV Oregon St. Eldorado NV St. Louis Rams Draft: St. Louis Rams in the 1st round (24th overall) of the 2004 NFL Draft.
1989-09-08 6 2 210 
    in Dallas, TX Baylor W.T. White TX Dallas Cowboys Draft: Dallas Cowboys in the 3rd round (74th overall) of the 2013 NFL Draft.
1989-09-08 6 2 210 
    in Da

In [29]:
### This block is for the players who have the same name as another, the links were also manually pulled 
### Note: the player id is included for these players because there needs to be a way to merge them 
data = []
for i in range(len(duplicates)):

    player = duplicates['player_name'][i]
    link = duplicates['link'][i]
    player_id = duplicates['player_id'][i]
    
    # Check if link is a valid URL
    if link.startswith('http'):
        # Retrieve HTML content from the response
        source_code = requests.get(link)
        
        # Check if the request was successful
        if source_code.status_code == 200:
            source_code = source_code.text
            player_info_scrape(source_code, player, player_id)
            #data.append(player_id)
        else:
            print(f"Failed to fetch {link}, status code: {source_code.status_code}")
    else:
        print(f"Invalid URL: {link}") 

        
df = pd.DataFrame(np.array(data))
df.columns = "player_id player_name birthday feet inches weight born college high_school_name high_school_state draft_team draft_info".split()
print(df)
df.to_csv("duplicate_player_info_scrape.csv", mode = 'w', header = True, index = False)

1975-08-14 6 0 218 
    in New Orleans, LA Fresno St. Mira Mesa CA Arizona Cardinals Draft: Arizona Cardinals in the 4th round (95th overall) of the 1998 NFL Draft.
1997-10-05 6 4 223 
    in Woodland Hills, CA USC Oaks Christian CA Indianapolis Colts Draft: Indianapolis Colts in the 2nd round (34th overall) of the 2020 NFL Draft.
1984-01-04 6 5 235 
    in Tampa, FL USC Plant FL Detroit Lions Draft: Detroit Lions in the 1st round (10th overall) of the 2005 NFL Draft.
1987-05-18 6 2 204 
    in Buffalo, NY Syracuse Riverside NY Tampa Bay Buccaneers Draft: Tampa Bay Buccaneers in the 4th round (101st overall) of the 2010 NFL Draft.
1994-10-04 6 4 218 
    in Vance, SC Clemson Lake Marion SC Los Angeles Chargers Draft: Los Angeles Chargers in the 1st round (7th overall) of the 2017 NFL Draft.
1983-04-22 6 6 222 
    in Fort Smith, AR Arkansas Van Buren AR Jacksonville Jaguars Draft: Jacksonville Jaguars in the 1st round (21st overall) of the 2005 NFL Draft.
1993-03-07 6 2 239 
    in Sef